<a href="https://colab.research.google.com/github/Elipro2289/IA-Proyecto/blob/main/CatVSDog.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

#Fase 1. Fases de interferencia
Importamos librerías necesitadas para esta parte del código, luego forzamos que nuestro entorno de ejecución use un modelo de keras anterior, ya que si no, el programa no podrá ejecutarse correctamente.
Posteriormente se carga el modelo de inteligencia artificial, pero con compile=False ya que no necesitamos compilar el modelo ahora mismo.
Por último, el programa lee las etiquetas que nosotros hemos asignado a cada clase, en este caso perro/gato.

In [9]:
import os
import numpy as np
# 1. Activamos el modo legacy (Keras 2)
os.environ['TF_USE_LEGACY_KERAS'] = '1'
from tensorflow import keras



#Carga el modelo
mi_modelo = keras.models.load_model("/content/drive/MyDrive/Progra/keras_model.h5", compile=False)

# Carga las etiquetas de las clases
nombre_clases = open("/content/drive/MyDrive/Progra/labels.txt", "r").readlines()

print(nombre_clases)

['0 Cat\n', '1 Dog\n']


#Fase 2. Adquirir datos
Aquí se selecciona una sola imagen que será clasificada por el modelo.


In [10]:
from PIL import Image
imagen= Image.open("/content/drive/MyDrive/Progra/test/cat.9862.jpg").convert("RGB") #Esta parte del código se asegura de que la imagen este en formato RGB.

#Fase 3.
En este paso se pre-procesa la imagen, redimensionandola a 224x224 pixeles, para posteriormente convertirla en un array de NumPy y normalizar los valores de los píxeles para que estén en el rango de -1 a 1, que es lo espera el modelo de inteligencia artificial.


In [11]:
from PIL import ImageOps
size = (224, 224)
image = ImageOps.fit(imagen, size, Image.Resampling.LANCZOS)
imagen_array = np.asarray(image)
normalizada_imagen_array = (imagen_array.astype(np.float32) / 127.5) - 1

#Fase 4
Se toma la imagen deseada y se introduce dentro de un lote de imagenes, para entrenar de forma correcta al modelo. (Posteriormente se le añadirán todas las imagenes de la carpeta test)

In [12]:
lote_imagenes = np.ndarray(shape=(1, 224, 224, 3), dtype=np.float32)
lote_imagenes[0] = normalizada_imagen_array
resultados= mi_modelo.predict(lote_imagenes)



1/1 [==============================] - 1s 750ms/step


#Fase 5.
 La Fase 5 interpreta los resultados del modelo, identificando la clase predicha con mayor probabilidad y mostrando esa etiqueta y su probabilidad. Esta parte del código se ve reutilizada en la fase 6, para clasificar todas las imágenes del lote.

In [13]:
indice = np.argmax(resultados[0])
etiqueta = nombre_clases[indice]
print("La imagen es de clase: ", etiqueta)
probabilidad = resultados[0][indice]
print("Con una probabilidad de: ", probabilidad)

La imagen es de clase:  0 Cat

Con una probabilidad de:  0.9999999


#Fase 6: Lectura de carpetas
Se recopilan etapas previas para hacer que el modelo sea capaz de leer toda la carpeta de test a la vez, sin necesidad de tener que poner todas las imagenes por separado. Las fases repetidas son la 2, 3, 4 y 5.

In [14]:
import os
import numpy as np
from PIL import Image, ImageChops # Se importan librerías necesarias para el código

lista_archivos = os.listdir("/content/drive/MyDrive/Progra/test")
total_predicciones = 0
aciertos = 0
media_probabilidad_aciertos = 0.0
# predicciones incorrectas contendrá una lista con los nombres de las imágenes mal clasificadas
predicciones_incorrectas = []

def predecir_imagen(ruta_imagen):
    #Esto es el proceso que se utilizó en la fase 2, pero esta vez definido dentro de la función predecir_imagen
    imagen = Image.open(ruta_imagen).convert("RGB")

    #Esto es el proceso que se utilizó en la fase 3, pero esta vez definido dentro de la función predecir_imagen
    size = (224, 224)
    image = ImageOps.fit(imagen, size, Image.Resampling.LANCZOS)
    imagen_array = np.asarray(image)
    normalizada_imagen_array = (imagen_array.astype(np.float32) / 127.5) - 1

    #Esto es el proceso que se utilizó en la fase 4, pero esta vez definido dentro de la función predecir_imagen
    lote_imagenes = np.ndarray(shape=(1, 224, 224, 3), dtype=np.float32)
    lote_imagenes[0] = normalizada_imagen_array
    resultados = mi_modelo.predict(lote_imagenes)

    #Esto es el proceso que se utilizó en la fase 5, pero esta vez definido dentro de la función predecir_imagen
    indice = np.argmax(resultados[0])
    etiqueta = nombre_clases[indice].strip() # Esto sirve para eliminar el carácter de nueva línea.
    probabilidad = resultados[0][indice]

    return etiqueta, probabilidad


for nombre_archivo in lista_archivos:
    if "cat" in nombre_archivo:
        etiqueta_esperada = "0 Cat"
    elif "dog" in nombre_archivo:
        etiqueta_esperada = "1 Dog"
    else:
        continue  # Saltar archivos que no sean de gatos o perros

    total_predicciones += 1

    ruta_imagen = os.path.join("/content/drive/MyDrive/Progra/test", nombre_archivo)

    # Aquí obtendriamos la predicción del modelo llamando a una función que
    # implemente las fases de inferencia
    etiqueta_predicha, probabilidad = predecir_imagen(ruta_imagen)

    if etiqueta_predicha == etiqueta_esperada:
        aciertos += 1
        media_probabilidad_aciertos += probabilidad
    else:
        info_error= {"archivo": nombre_archivo, "prediccion": etiqueta_predicha,
             "probabilidad": probabilidad}
        predicciones_incorrectas.append(info_error)




1/1 [==============================] - 0s 20ms/step


#Fase 7.
Cálculo de métricas y generación del informe. Esta fase final, se encarga de entregar los datos de las predicciones de una forma útil.

In [15]:
precision = aciertos / total_predicciones if total_predicciones > 0 else 0
media_probabilidad_aciertos /= aciertos if aciertos > 0 else 1
informe = f"""
Informe de Evaluación del Modelo

Total de predicciones: {total_predicciones}
Aciertos: {aciertos}
Precisión: {precision:.2%}
Probabilidad media de aciertos: {media_probabilidad_aciertos:.2%}

Predicciones incorrectas:
"""

for error in predicciones_incorrectas:
    informe += f"Archivo: {error['archivo']}, Predicción: {error['prediccion']}, Probabilidad: {error['probabilidad']:.2%}\n"

print(informe)


Informe de Evaluación del Modelo

Total de predicciones: 400
Aciertos: 391
Precisión: 97.75%
Probabilidad media de aciertos: 99.11%

Predicciones incorrectas:
Archivo: dog.9804.jpg, Predicción: 0 Cat, Probabilidad: 83.01%
Archivo: dog.9833.jpg, Predicción: 0 Cat, Probabilidad: 97.00%
Archivo: dog.9868.jpg, Predicción: 0 Cat, Probabilidad: 99.85%
Archivo: dog.9911.jpg, Predicción: 0 Cat, Probabilidad: 85.84%
Archivo: dog.9935.jpg, Predicción: 0 Cat, Probabilidad: 100.00%
Archivo: dog.994.jpg, Predicción: 0 Cat, Probabilidad: 59.56%
Archivo: cat.9947.jpg, Predicción: 1 Dog, Probabilidad: 90.80%
Archivo: cat.9943.jpg, Predicción: 1 Dog, Probabilidad: 52.35%
Archivo: cat.9960.jpg, Predicción: 1 Dog, Probabilidad: 74.86%

